In [381]:
# Load dependencies
import csv
import pandas as pd
import numpy as np
import os
import gc
import warnings
import itertools as it
import DataProcessFunctions as DP
import PredictionStep1 as pred
import SupportFunctions as supp
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression as lr
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from tensorflow.keras.layers import Dense
from matplotlib.pyplot import cm


%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True)
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Initialize warning log container 
log = list()

In [3]:
# Load Pre-processed data from Data Processing PCA.ipynb
FM_data = pd.read_csv(os.path.dirname(os.getcwd()) + '\\FM2_data.csv')
returns = pd.read_csv(os.path.dirname(os.getcwd()) + '\\returns2_data.csv').set_index(["permno", "date"])
industry_code = pd.read_csv(os.path.dirname(os.getcwd()) + '\\industry2_codes.csv').set_index(["permno", "date"])

supp.downcast(FM_data)
supp.downcast(returns)
supp.downcast(industry_code)

Before downcast: 1.769 GB and float64    102
int64        3
dtype: int64
After downcast: 0.878 GB and float32    102
int32        2
int8         1
dtype: int64
Before downcast: 0.026 GB and float64    1
dtype: int64
After downcast: 0.018 GB and float32    1
dtype: int64
Before downcast: 0.026 GB and float64    1
dtype: int64
After downcast: 0.018 GB and float32    1
dtype: int64


In [4]:
# Set number of iterations
ite = 29 # 30 splits --> iteration = [0;29]

# Initialize arrays to store loss and explained variation
loss = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])
xplained_variation = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])

# Initialize arrays to store annual loss and explained variation
loss_annual = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])
xplained_variation_annual = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])

# Initialize arrays to store predicted and actual returns used in portfolio sorts later
LR_pred_actual = pd.DataFrame()
lasso_pred_actual = pd.DataFrame()
NN_pred_actual = pd.DataFrame()

for i in range(ite):
    
    # Compute training, validation, and test set
    training, validation, test = DP.complete_data_process(industry_code, returns, FM_data, iteration = i)
    
    # Split in X and Y
    training_x, training_y = pred.XY_split(training)
    validation_x, validation_y = pred.XY_split(validation)
    test_x, test_y = pred.XY_split(test)
    
    # Algorithm 1: Simple Linear (PCR in Gu, kelly, and Xiu (2020) due to PCA)
    # Fit model on training set & predict on test set
    LR = lr().fit(training_x, training_y)
    LR_pred = LR.predict(test_x)
    
    # Algoritm 1: Compute loss and explained varation, and combine 
    # actual and predicted returns. Append all. 
    LR_loss, LR_explained_var, LR_pred_actual_temp, LR_loss_annual, LR_explained_var_annual = pred.to_append(LR_pred, test_y)
    loss.iloc[0, i] = LR_loss
    loss_annual.iloc[0, i] = LR_loss_annual
    xplained_variation.iloc[0, i] = LR_explained_var
    xplained_variation_annual.iloc[0, i] = LR_explained_var_annual
    LR_pred_actual = LR_pred_actual.append(LR_pred_actual_temp)
    
    
    # ALgorithm 2: LASSO
    # Fit model on training set and select tuning parameter based on validation set
    lambda_grid = pred.lambda_grid(training_x, training_y)
    loss_validation = []

    for lamb in lambda_grid:
        lasso = Lasso(alpha = lamb, tol = 0.001).fit(training_x, training_y)
        lasso_pred = lasso.predict(validation_x)
        loss_validation.append(pred.loss_function(lasso_pred, validation_y))

    # Fit model with error minimizing tuning parameter
    lambda_min = lambda_grid[loss_validation.index(min(loss_validation))]
    lasso_min = Lasso(alpha = lambda_min).fit(training_x, training_y)
    lasso_min_pred = lasso_min.predict(test_x)

    # Algorithm 2: Appending 
    lasso_loss, lasso_explained_var, lasso_pred_actual_temp, lasso_loss_annual, lasso_explained_var_annual = pred.to_append(lasso_min_pred, test_y)
    loss.iloc[1, i] = lasso_loss
    loss_annual.iloc[1, i] = lasso_loss_annual
    xplained_variation.iloc[1, i] = lasso_explained_var
    xplained_variation_annual.iloc[1, i] = lasso_explained_var_annual
    lasso_pred_actual = lasso_pred_actual.append(lasso_pred_actual_temp)
    
    
    # ALgorithm 3: NN 
    # Arkitektur skal gøres uden for loopet! Skal jeg gøre NN mere avanceret? Jeg har stadig linear som de sidste lag også 
    # Build NN architecture 
    model = tf.keras.models.Sequential()
    model.add(Dense(units = 32, activation = 'relu', input_dim = len(training_x.columns))) # 1. Hidden layer
    model.add(Dense(units = 16, activation = 'relu')) # 2. Hidden layer
    model.add(Dense(units = 8, activation = 'relu')) # 3. Hidden layer
    model.add(Dense(units = 4, activation = 'relu')) # 4. Hidden layer
    model.add(Dense(units = 1, activation = 'linear')) # Output layer - linear fixes nonnegative predictions

    # Compile model
    model.compile(optimizer = 'adam', loss = 'mean_squared_error') # jeg kan også bruge huber loss ved 'huber'

    # Fit model
    model.fit(training_x, training_y, epochs = 1, batch_size = 40, verbose = 1)

    # Compute predictions
    NN_pred = model.predict(test_x)
    
    # Appending
    NN_loss, NN_explained_var, NN_pred_actual_temp, NN_loss_annual, NN_explained_var_annual = pred.to_append(NN_pred, test_y)
    loss.iloc[2, i] = NN_loss
    loss_annual.iloc[2, i] = NN_loss_annual
    xplained_variation.iloc[2, i] = NN_explained_var
    xplained_variation_annual.iloc[2, i] = NN_explained_var_annual
    NN_pred_actual = NN_pred_actual.append(NN_pred_actual_temp)
    
    del training
    del validation
    del test
    del training_x
    del training_y
    del validation_x
    del validation_y
    del test_x
    del test_y
    
    gc.collect()

In [329]:
# Return for each decile at all points in time
LR_step_1 = pred.portfolio_sorts_1(LR_pred_actual)
lasso_step_1 = pred.portfolio_sorts_1(lasso_pred_actual)
NN_step_1 = pred.portfolio_sorts_1(NN_pred_actual)

# Accumualtive return of all deciles for both
# predicted and actual returns 
LR_cum = pred.portfolio_sorts_acc_return(LR_step_1)
lasso_cum = pred.portfolio_sorts_acc_return(lasso_step_1)
NN_cum = pred.portfolio_sorts_acc_return(NN_step_1)

# Monthly average return and std. deviation, and annualized SR
# of both predicted and actual returns
LR_mean, LR_std, LR_sr = pred.portfolio_sorts_SR(LR_step_1)
lasso_mean, lasso_std, lasso_sr = pred.portfolio_sorts_SR(lasso_step_1)
NN_mean, NN_std, NN_sr = pred.portfolio_sorts_SR(NN_step_1)

In [339]:
# Figure: Cumulative return of all deciles for each ML model
pred.cumulative_ret_fig(data = LR_step_1, name = "LR_cumulative_ret", save_fig = True, hide = True)
pred.cumulative_ret_fig(data = lasso_step_1, name = "lasso_cumulative_ret", save_fig = True, hide = True)
pred.cumulative_ret_fig(data = NN_step_1, name = "NN_cumulative_ret", save_fig = True, hide = True)

In [408]:
# Figure: Cumulative return of 1st and 10th decile of specified ML models 
pred.deciles_10_1_fig(name = "deciles_10_1", save_fig = True, hide = True, data1 = LR_step_1, data2 = lasso_step_1, data3 = NN_step_1)

In [482]:
# Prep data for export to R so as to customaize for tables 

# Table 1: monthly loss and explained variation 
monthly_pricing_error = loss.mean(axis = 1)
monthly_xplained_variation = xplained_variation.mean(axis = 1)
table1 = pd.concat([monthly_pricing_error, monthly_xplained_variation], axis = 1)
table1.columns = ["Squared Pricing Error", "Explained Variation"]
table1 = table1.transpose()
table1.columns = ["Linear Regression", "Lasso", "Neural Network"]
table1.to_csv(os.path.dirname(os.getcwd()) + '\\table1_data.csv', header = True, index = True)

# Table 2: Annual loss and explained variation
annual_pricing_error = loss_annual.mean(axis = 1)
annual_xplained_variation = xplained_variation_annual.mean(axis = 1)
table2 = pd.concat([annual_pricing_error, annual_xplained_variation], axis = 1)
table2.columns = ["Squared Pricing Error", "Explained Variation"]
table2 = table2.transpose()
table2.columns = ["Linear Regression", "Lasso", "Neural Network"]
table2.to_csv(os.path.dirname(os.getcwd()) + '\\table2_data.csv', header = True, index = True)

# Table 3:
table3_LR = pd.concat([LR_mean, LR_std.ret, LR_sr.ret], axis = 1).round(3)
table3_LR.columns = ["Avg", "Pred", "Std", "SR"]
table3_lasso = pd.concat([lasso_mean, lasso_std.ret, lasso_sr.ret], axis = 1).round(3)
table3_lasso.columns = ["Avg", "Pred", "Std", "SR"]
table3_NN = pd.concat([NN_mean, NN_std.ret, NN_sr.ret], axis = 1).round(3)
table3_NN.columns = ["Avg", "Pred", "Std", "SR"]

# Table 4: Tabel for appendix figurerne (skal bruge LR_cum, lasso_cum, NN_cum) og så bare kun ret søjlen. Har så tabel der viser end point for figurene (denne tabel kommer i appendix) 
table4 = pd.concat([LR_cum.ret, lasso_cum.ret, NN_cum.ret], axis = 1).round(3)
table4.columns = ["Linear Regression", "Lasso", "Neural Network"]
table4 = table4.transpose()
table4.to_csv(os.path.dirname(os.getcwd()) + '\\table4_data.csv', header = True, index = True) 

In [480]:
table4

decile,1,2,3,4,5,6,7,8,9,10
Linear Regression,-0.048,0.646,1.095,1.349,1.738,1.890,1.948,2.122,2.896,4.614
Lasso,0.257,0.589,1.153,1.284,1.723,2.057,2.123,2.595,2.722,3.748
Neural Network,0.927,1.380,1.289,1.650,1.742,1.847,2.165,2.098,2.271,2.881


In [ ]:
# Tror jeg skal dele portfolio sorts ind i 2 dele (2 funktinoer). Jeg har den første nu, som for hver periode hver decil spytter returns ud. Derfra skal jeg så bruge det output til 2 forskellige ting.
# 1) Jeg skal akkumulere returns over perioden for hver decil, så jeg ender med at kumulativt return for hver decil, i.e. 10 tal
# 2) For at lave figur A.7 i gu, kelly xiu appendix + for at regne sharpe ratio ud, så skal jeg bruge returns for hver decil i hver periode. For at lave figuren skal jeg til hver periode akkumulere returns a la hvad jeg 
# gjorde for PCA figuren. For at regne sharpe ratio ud skal jeg for hver decil over perioden finde average return og std. dev. 

In [ ]:
# Portfolio sorts; groupby dato. for hver gruppe splitter vi nu preds op i deciler og calculater return for hver portefølje (på det faktiske return). Vi gør dette for hver måned og så får vi så efter et år 12 observationer
# for hver portefølje, vi kan nu calculate sharpe ratio for hver a porteføljerne (mobthly sharpe ratio) baseret på det ene år, hvis vi nu har længere periode kan jeg compoute monthly sharpe ratio på den længere periode, man kan så
# annualize ved at gange med sqrt(12). 

# hvad siger gu kelly og xiu om portfolio sorts hvad gør de? 

# det kunne være sjovt at sammenligne med konventielle asset pricing modeller som f.eks. FF3 og CAPM -- gør de vel egentlig ogsåi  gu kelly og xiu 


# jeg skal have ryddet op i koden med loopet. både NN struktur men også bare resten generelt 
# Skal jeg lave en normals OLS på ikke PCA data? bare som totalt benchmark? 

# jeg skal lave gu kelly xiu tabel 7 -- er der andre tabeller jeg skal have med? Jeg tænker at eksportere relevant data (i korrekt format) til R, så jeg kan bruge table-theme jeg har lavet der